In [13]:
import pandas as pd
import numpy as np
import sqlite3

In [14]:
df = pd.read_csv('../database/alunos-ativos-da-pos-graduacao.csv', sep=';', usecols=['nome_aluno']).sample(frac=1).reset_index(drop=True)
df = df.rename(columns={'nome_aluno' : 'nome'})

In [15]:
#Atribuindo sexo considerando nomes com maior frequencia
df_nomes_frequencia = df['nome'].map(lambda y: y.split()[0]).value_counts()

nomes_masculinos = ['Lucas', 'João', 'Gabriel', 'Matheus', 'Rafael', 'Guilherme', 'Gustavo', 'Bruno', 'Luiz', 'Paulo', 'Pedro', 'André', 'José', 'Rodrigo', 'Thiago', 'Felipe', 'Daniel', 'Marcelo', 'Eduardo']
nomes_femininos = ['Ana', 'Maria', 'Mariana', 'Bruna', 'Gabriela', 'Camila', 'Fernanda', 'Amanda', 'Aline', 'Leticia', 'Larissa', 'Juliana', 'Jéssica', 'Beatriz', 'Marina']

df['sexo'] = None

df.loc[df['nome'].str.split().str[0].isin(nomes_masculinos), 'sexo'] = 'M'
df.loc[df['nome'].str.split().str[0].isin(nomes_femininos), 'sexo'] = 'F'

lista_sexo_aleatorio = np.random.choice(['M', 'F'], size=df['sexo'].isnull().sum())
df.loc[df['sexo'].isnull(), 'sexo'] = lista_sexo_aleatorio

In [16]:
#Define data de nascimento e idade
df['nascimento'] = None
df.loc[df['nascimento'].isnull(), 'nascimento'] = np.random.choice(pd.date_range(start='1910-01-01', end='2019-12-31').strftime('%Y-%m-%d'), size=len(df))

df['idade'] = (pd.Timestamp.now() - pd.to_datetime(df['nascimento'])).dt.days // 365

In [17]:
#define altura com influencia da idade e sexo
df['altura'] = np.random.randint(120, 210, size=len(df))
df.loc[df['idade'] <= 18, 'altura'] -= (25 * (1 - (df['idade'] / 18) ** 2 )).round() #quanto menor a idade, maior o decrescimo
df.loc[df['idade'] >= 60, 'altura'] -= (30 * ((df['idade'] - 60) / (df['idade'].max() - 60)) ** 2).round() #quanto maior a idade, maior o decrescimo

df.loc[df['sexo'] == 'F', 'altura'] -= np.random.randint(1, 8)
df.loc[df['sexo'] == 'M', 'altura'] += np.random.randint(1, 8)

In [18]:
#define o peso com influencia da idade e sexo
df['peso'] = np.round(np.random.uniform(25.0, 150, size=len(df)), decimals=2)
df.loc[df['idade'] <= 18, 'peso'] -= (8 * (1 - (df['idade'] / 18) ** 2 )).round(decimals=2)
df.loc[df['idade'] >= 60, 'peso'] -= (15 * ((df['idade'] - 60) / (df['idade'].max() - 60)) ** 2).round(decimals=2)

df.loc[df['sexo'] == 'F', 'peso'] -= (df['peso'] * 0.05).round(decimals=2)
df.loc[df['sexo'] == 'M', 'peso'] += (df['peso'] * 0.05).round(decimals=2)

In [19]:
# define pressao arterial
df['pressao-arterial-sistolica'] = 10 * np.random.randint(5, 23, size=len(df))
df['pressao-arterial-diastolica'] = 10 * np.random.randint(3, (df['pressao-arterial-sistolica']/10), size=len(df))

df.loc[df['idade'] <= 18, ['pressao-arterial-sistolica', 'pressao-arterial-diastolica']] -= (10 * np.random.randint(1, 4))

In [20]:
# define glicemia com aumento na terceira idade
df['glicemia'] = 5 * np.random.randint(8, 44, size=len(df))
df.loc[df['idade'] >= 60, 'glicemia'] += 5 * np.random.randint(1, 4)

In [21]:
# Gera um valor de frequencia cardiaca com influencia da idade
df['frequencia-cardiaca'] = np.random.randint(30, 210, size=len(df))

df.loc[df['idade'] <= 18, 'frequencia-cardiaca'] += (np.random.randint(20, 40) * (1 - (df['idade'] / 18) ** 2 )).round()
df.loc[df['idade'] >= 60, 'frequencia-cardiaca'] += (np.random.randint(10, 30) * ((df['idade'] - 60) / (df['idade'].max() - 60)) ** 2).round()

df

,nome,sexo,nascimento,idade,altura,peso,pressao-arterial-sistolica,pressao-arterial-diastolica,glicemia,frequencia-cardiaca
0,Letícia Gabriele Ribeiro Rocumba,M,1961-03-11,63,194,45.80,90,40,205,64
1,Diego Oliveira Lira,M,1988-05-08,36,128,41.75,130,80,160,176
2,Heitor Menezes Gomes,F,1923-02-05,101,133,22.49,50,40,90,189
3,Monithielly Regina Zamboni,F,1965-07-06,59,143,138.18,60,40,190,134
4,Clóvis Hiran Fuentes Mauá Filho,F,1925-07-25,99,184,69.00,130,30,110,51
...,...,...,...,...,...,...,...,...,...,...
3880,Nadia Elisa Gonçalves,M,1920-01-22,104,180,143.01,170,40,165,93
3881,Sandra Batista Medeiros,F,1971-11-25,52,146,39.99,180,70,135,108
3882,Maurício Dutra Félix,F,1944-07-02,80,166,138.76,100,80,80,161
3883,Cristiano Parra Duarte,M,1927-03-10,97,109,110.66,110,80,65,164


In [22]:
conexao = sqlite3.connect('../mysite/db.sqlite3')

df.to_sql('Pacientes', conexao, if_exists='replace', index=False)

3885

In [23]:
"""
df['circunferencia-cintura']
df['diabetes']
df['doenca-cronica']
df['tabagismo']
df['colesterol']
df['funcao-renal']
df['alergia']
df['duracao-sono']"""

"\ndf['circunferencia-cintura']\ndf['diabetes']\ndf['doenca-cronica']\ndf['tabagismo']\ndf['colesterol']\ndf['funcao-renal']\ndf['alergia']\ndf['duracao-sono']"